In [34]:
from sklearn.linear_model import LogisticRegression
import requests
import json
import time
import time 
import sqlalchemy as sqla
from sqlalchemy import create_engine
import csv
import pandas as pd
import numpy as np

#URI = "dublinbikesdatabase.ck8yyafvjq4p.eu-west-1.rds.amazonaws.com"
#PORT = "3306"
#DB = "dbbikes"
#USER = "Group26"
#PASSWORD = "Grouptwentysix"

#engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [ ]:
#conn = engine.connect()

#result = conn.execute("SELECT * FROM availability")

#data = result.fetchall()

#conn.close()#

In [ ]:
#with open("availability.csv", "w", newline="") as f:
    #writer = csv.writer(f)
    #writer.writerows(data)

In [ ]:
#conn = engine.connect()

#result = conn.execute("SELECT * FROM weather")

#data = result.fetchall()

#conn.close()

In [ ]:
#with open("weather.csv", "w", newline="") as f:
    #writer = csv.writer(f)
    #writer.writerows(data)

In [35]:
dfa = pd.read_csv('availability.csv', names=['number', 'available_bikes', 'available_bike_stands', 'time'])

In [36]:
dfw = pd.read_csv('weather.csv', names=['id', 'main', 'description', 'temp', 'feels_like', 'humidity', 'visibility',
                                            'speed','deg','all','dt'])

In [37]:
dfw.drop(['humidity','visibility','deg','all','description','deg','all','feels_like'], inplace=True, axis=1)

In [ ]:
startTime = 1677668400
endTime = 1680771600
times = np.linspace(startTime,endTime,863)

In [38]:
station_data = {}
grouped = dfa.groupby('number')

for name, group in grouped:
    new_station_df = pd.DataFrame(group, columns=['number','available_bikes','available_bike_stands','time'])
    station_data[name] = new_station_df

station_data = {f'df{key}': value for key, value in station_data.items()}
for name, df in station_data.items():
    globals()[name] = df

In [39]:
dfw['dt'] = pd.to_datetime(dfw['dt'], unit='s')
dfw['dt'] = dfw['dt'].dt.floor('H') # Drop to nearest hour
dfw['dt'] = dfw['dt'].apply(lambda x: int(x.timestamp()))

dfa['time'] = pd.to_datetime(dfa['time'], unit='s')
dfa['time'] = dfa['time'].dt.floor('H') # Drop to nearest hour
dfa['time'] = dfa['time'].apply(lambda x: int(x.timestamp()))
dfa.tail(10)

,number,available_bikes,available_bike_stands,time
1174535,3,10,10,1680768000
1174536,40,15,6,1680768000
1174537,29,3,26,1680768000
1174538,103,4,36,1680768000
1174539,28,19,11,1680768000
1174540,39,0,20,1680768000
1174541,83,13,27,1680768000
1174542,92,25,15,1680768000
1174543,21,10,20,1680768000
1174544,88,14,15,1680768000


In [40]:
weather_data = {}
grouped = dfw.groupby('dt')

for name, group in grouped:
    new_station_df = pd.DataFrame(group, columns=['id','main','temp','speed','dt'])
    weather_data[name] = new_station_df

weather_data = {f'df{key}': value for key, value in weather_data.items()}
for name, df in weather_data.items():
    globals()[name] = df

In [73]:
cleanedWeather = pd.DataFrame(columns=['Time', 'Weather', 'Temp', 'Wind Speed'])

for key, value in weather_data.items():
    mode_value = value['main'].mode()[0]
    temp_mean = round(value['temp'].mean(),2)
    speed_mean = round(value['speed'].mean(),2)
    key = key[2:]
    cleanedWeather.loc[len(cleanedWeather)] = [key, mode_value, temp_mean, speed_mean]

In [74]:
cleanedWeather

,Time,Weather,Temp,Wind Speed
0,0,Clouds,281.03,6.85
1,1677682800,Clouds,280.91,6.27
2,1677686400,Clouds,280.80,6.09
3,1677690000,Clouds,280.53,6.31
4,1677693600,Clouds,280.23,4.80
...,...,...,...,...
854,1680753600,Rain,280.46,4.68
855,1680757200,Clouds,280.50,3.56
856,1680760800,Rain,280.70,3.79
857,1680764400,Clouds,280.63,4.07


In [ ]:
X = get_features(station=station_id)
y = get_availability(station=station_id)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# fit final model
model[station_id] = LogisticRegression() # choose your own model here...
model[station_id].fit(X_train, y_train)

# evaluate...
# define one new instance
Xnew = [[date, time, wind, temp, ...]] # query features
# make a prediction
ynew = model[station_id].predict(Xnew)
print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))